Swagger codegen is so broken it's amazing

```

# java 10 doesn't work, use java8 because reflection issues (??)
sudo apt install openjdk-8-jdk-headless
export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64/
export PATH=${JAVA_HOME}/bin:$PATH # not sure if needed

# build it
git clone https://github.com/swagger-api/swagger-codegen
cd swagger-codegen
mvn clean package

# generate client
cd ../eve_market_analysis
java -jar ../swagger-codegen/modules/swagger-codegen-cli/target/swagger-codegen-cli.jar \
    generate -i https://esi.evetech.net/latest/swagger.json -l python --library asyncio -o python-client

# fix yet another bug
find python-client/ -name \*.py | xargs sed -i 's/\<async:/async_:/g'
find python-client/ -name \*.py | xargs sed -i 's/\<async=/async_=/g'

# conda install
source activate eve
cd python-client
python3 setup.py install
```

In [1]:
%%capture
""" only run this once

from notebook.services.config import ConfigManager
c = ConfigManager()
c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})
"""

In [2]:
import swagger_client as eve
import asyncio

In [3]:
class ESIClient:
    def __init__(self, cls):
        self.cls = cls
    
    async def __aenter__(self):
        self.client = self.cls(eve.ApiClient())
        # not sure if this actually works
        self.client.api_client.user_agent = "nickgarvey Eve Dev"
        return self.client

    async def __aexit__(self, exc_type, exc, tb):
        # generated code don't close the pool_manager, so do it ourselves
        await self.client.api_client.rest_client.pool_manager.close()


async def gen_region(universe_client, region_id):
    return await universe_client.get_universe_regions_region_id(region_id)


async def gen_all_regions():
    async with ESIClient(eve.UniverseApi) as universe_client:
        region_ids = await universe_client.get_universe_regions()
        regions = await asyncio.gather(*[gen_region(universe_client, i) for i in region_ids[:3]])
    return regions


loop = asyncio.new_event_loop()
regions = loop.run_until_complete(gen_all_regions())
regions = {r.region_id: r for r in regions}

In [4]:
async def gen_all_market_data(regions):
    async with ESIClient(eve.MarketApi) as market_client:
        print(help(market_client.get_markets_region_id_history))
        
loop = asyncio.new_event_loop()
markets = loop.run_until_complete(gen_all_market_data(dict(list(regions.items())[:3])))
import pandas

Help on method get_markets_region_id_history in module swagger_client.api.market_api:

get_markets_region_id_history(region_id, type_id, **kwargs) method of swagger_client.api.market_api.MarketApi instance
    List historical market statistics in a region  # noqa: E501
    
    Return a list of historical market statistics for the specified type in a region  --- Alternate route: `/dev/markets/{region_id}/history/`  Alternate route: `/legacy/markets/{region_id}/history/`  Alternate route: `/v1/markets/{region_id}/history/`  --- This route expires daily at 11:05  # noqa: E501
    This method makes a synchronous HTTP request by default. To make an
    asynchronous HTTP request, please pass async_=True
    >>> thread = api.get_markets_region_id_history(region_id, type_id, async_=True)
    >>> result = thread.get()
    
    :param async bool
    :param int region_id: Return statistics in this region (required)
    :param int type_id: Return statistics for this type (required)
    :param str